# Installing dependencies

In [ ]:
# !py -m pip install pyvista --upgrade
# !py -m pip install ipympl --upgrade
# !py -m pip install trame --upgrade
# !py -m pip install scipy --upgrade
# !py -m pip install shapely -- upgrade
# !py -m pip install tqdm -- upgrade

# Imports

In [1]:
import pyvista as pv
import numpy as np
import math

import matplotlib.pyplot as plt
import imageio

from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy import optimize
from scipy.special import comb
from scipy.interpolate import interp1d

from math import sqrt, cos, sin, pi, atan2, degrees, floor

import shapely as sp
from shapely import affinity, distance
from shapely.geometry import Polygon, Point, LinearRing, LineString, MultiPoint
from shapely.plotting import plot_polygon, plot_points, plot_line
from shapely.ops import transform, nearest_points, split
from shapely.affinity import rotate

from tqdm.notebook import tqdm

# Input parameters

In [263]:
R = 75                  # radius of disk
n = 4000                # number of points along contours
r = R+2                 # initial ordinate of disk center

contain_percent = 0.7
max_delta_distance = 0.08
disk_delta_translation = max_delta_distance/5

rough_filename = "data\\B4969-B_PLANNING_pear\\rough.obj"
diamond_filename = "data\\B4969-B_PLANNING_pear\\diamond.obj"

framerate = 20
draw_path = 'algo/pics/'
movie_path = 'algo/'

# Obtaining contours

In [195]:
def matan(point):
    angle = atan2(point.y, point.x)
    if angle <= 0: angle += 2*pi
    return angle

def norma(point):
    return Point(0,0).distance(point)

def Swap(val1, val2):
    val1 = val1 + val2
    val2 = val1 - val2
    val1 = val1 - val2
    return val1, val2

def GetContourFromOBJ(filename):
    return LinearRing(MultiPoint(np.delete(pv.read(filename).project_points_to_plane().points, 2, 1)).convex_hull.boundary.coords)

def GetCornerIndices(pts):
    al = []
    for i in range(len(pts)):
        al.append([pts[i].x, pts[i].y])

    a = np.array(al)
    dx_dt = np.gradient(a[:, 0])
    dy_dt = np.gradient(a[:, 1])
    velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

    ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

    tangent = np.array([1/ds_dt]).transpose() * velocity

    tangent_x = tangent[:, 0]
    tangent_y = tangent[:, 1]

    deriv_tangent_x = np.gradient(tangent_x)
    deriv_tangent_y = np.gradient(tangent_y)

    dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

    length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

    normal = np.array([1/length_dT_dt]).transpose() * dT_dt

    d2s_dt2 = np.gradient(ds_dt)
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)

    curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5
    t_component = np.array([d2s_dt2] * 2).transpose()
    n_component = np.array([curvature * ds_dt * ds_dt] * 2).transpose()

    acceleration = t_component * tangent + n_component * normal

    # plt.plot(curvature)
    return [idx for idx, val in enumerate(curvature) if val>=np.max(curvature)/2]

def ShiftConcatenate(data, shift):
    data = data.copy()
    for _ in range(shift):
        data.append(data.pop(0))
    return data

def GetSmoothedPear(pear):
    pear_original_pts, pts = [Point(pear.coords[i][0], pear.coords[i][1]) for i in range(len(pear.coords))], []
    for pt in pear_original_pts: pts.append([pt.x, pt.y])
    pts = sorted(pts, key=ClockwiseAngleAndDistance)
    pear_original__pts = [Point(pt[0], pt[1]) for pt in pts]
    corner_indices = GetCornerIndices(pear_original_pts)
    if len(corner_indices) > 0: 
        pear_original_shifted_pts = ShiftConcatenate(pear_original_pts, corner_indices[0])

    pear_original_shifted_pts, pts = list(set(pear_original_shifted_pts)), []
    for pt in pear_original_shifted_pts: pts.append([pt.x, pt.y])
    pts = sorted(pts, key=ClockwiseAngleAndDistance)
    pear_original_shifted_pts = [Point(pt[0], pt[1]) for pt in pts]

    corner_indices = GetCornerIndices(pear_original_shifted_pts)
    if len(corner_indices) > 0: 
        pear_original_shifted_pts = ShiftConcatenate(pear_original_shifted_pts, corner_indices[0])

    x, y = [pt.x for pt in pear_original_shifted_pts[1:]], [pt.y for pt in pear_original_shifted_pts[1:]]

    points = np.array([x, y]).T
    distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
    distance = np.insert(distance, 0, 0)/distance[-1]
    interpolator =  interp1d(distance, points, kind='cubic', axis=0)
    interpolated_points = interpolator(np.linspace(0, 1, 1000))
    pts = [Point(pt[0], pt[1]) for pt in interpolated_points]
    pts.append(pear_original_shifted_pts[0])

    return LinearRing(pts)

def GetBaloonedContour(contour, baloon_distance):
    return LinearRing(LineString(contour).offset_curve(baloon_distance, quad_segs=int(2**5), join_style=2, mitre_limit=5.0))

def ClockwiseAngleAndDistance(point):
    origin, refvec = [0,0], [1,0]
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # Return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

def GetOrderedIntersectionPoints(geom1, geom2):
    intersection, pts, spts = geom1.intersection(geom2), [], []
    if intersection.geom_type == 'LineString':
        return [], []
    else:
        for geom in intersection.geoms: pts.append([geom.x, geom.y])
        pts = sorted(pts, key=ClockwiseAngleAndDistance)
        for pt in reversed(pts): spts.append(Point(pt[0], pt[1]))
        angle1, angle2 = matan(spts[0]), matan(spts[1])
        line = LineString([(0,0), (30*cos((angle1+angle2)/2), 30*sin((angle1+angle2)/2))])
        pt1, pt2 = geom1.intersection(line), geom2.intersection(line)
        if norma(pt1) < norma(pt2): spts.append(spts.pop(0))
        return spts, [matan(pt) for pt in spts]

def GetSimpleOrderedIntersectionPoints(geom1, geom2):
    intersection, pts, spts = geom1.intersection(geom2), [], []
    if intersection.geom_type == 'LineString':
        return [], []
    elif intersection.geom_type == 'Point':
        return [intersection], [matan(intersection)]
    else:
        for geom in intersection.geoms: pts.append([geom.x, geom.y])
        pts = sorted(pts, key=ClockwiseAngleAndDistance)
        for pt in reversed(pts): spts.append(Point(pt[0], pt[1]))
        return spts, [matan(pt) for pt in spts]

def Contour2Points(contour, begin_angle, n):
    contour_pts = []
    for i in range(n):
        angle = begin_angle + (i/n)*2*pi
        intersection = contour.intersection(LineString([(0,0), (30*cos(angle), 30*sin(angle))]))
        if intersection.geom_type == 'Point': contour_pts.append(intersection)
        else: contour_pts.append(intersection.geoms[-1])
    return contour_pts

def GetNormalVectors(pts):
    al = []
    for i in range(len(pts)):
        al.append([pts[i].x, pts[i].y])

    a = np.array(al)
    dx_dt = np.gradient(a[:, 0])
    dy_dt = np.gradient(a[:, 1])
    velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

    ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

    tangent = np.array([1/ds_dt]).transpose() * velocity

    tangent_x = tangent[:, 0]
    tangent_y = tangent[:, 1]

    deriv_tangent_x = np.gradient(tangent_x)
    deriv_tangent_y = np.gradient(tangent_y)

    dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

    length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

    normal = np.array([1/length_dT_dt]).transpose() * dT_dt
    return [Point(-pt[0]/sqrt(pt[0]**2+pt[1]**2), -pt[1]/sqrt(pt[0]**2+pt[1]**2)) for pt in normal]

def GetRotatedReperIndices(angles, intersection_angles):
    indices = []
    local_angles = [angle-intersection_angles[0] for angle in intersection_angles]
    # indices.append(0)
    for angle in local_angles:
        for i in range(len(angles)):
            if angles[i] > angle:
                indices.append(i)
                break
    # indices.pop(-1)
    return indices, len(indices)

def GetReperIndices(angles, intersection_angles):
    indices = []
    for angle in intersection_angles:
        for i in range(len(angles)):
            if angles[i] > angle:
                indices.append(i)
                break
    return indices, len(indices)

def GetNormals(pts, blpts, n):
    normals = []
    for i in range(n):
        x, y = blpts[i].x-pts[i].x, blpts[i].y-pts[i].y
        norm = sqrt(x**2+y**2)
        normals.append(Point(x/norm, y/norm))
    return normals

def GetEdges(pts, n):
    edges = []
    for i in range(n):
        x, y = pts[(i+1)%n].x-pts[i].x, pts[(i+1)%n].y-pts[i].y

def GetBaloonedContourWithContainPercentage(geom1, geom2, n, contain_percent):
    geom1_pts, geom2_pts = Contour2Points(geom1, 0, n),  Contour2Points(geom2, 0, n)
    dsts = [geom1_pts[i].distance(geom2_pts[i]) for i in range(n)]
    hist, bin_edges = np.histogram(dsts, bins=floor(n/4))
    # plt.hist(dsts, bins=floor(n/4)); plt.show()
    sum, percent, whole_sum, i_edge = 0, 0, hist.sum(), 0
    for i in range(len(hist)):
        sum += hist[i]
        percent = sum / whole_sum
        if percent > contain_percent:
            i_edge = i-1
            break
    blgeom2 = GetBaloonedContour(geom2, np.min(dsts)+(i_edge/len(hist))*(np.max(dsts)-np.min(dsts)))
    spts, intersection_angles = GetOrderedIntersectionPoints(geom1, blgeom2)
    return blgeom2, spts

def GetMaxDistance(geom1_pts, geom2):
    dst_max = 0.0
    for i in range(len(geom1_pts)):
        pt1, pt2 = nearest_points(geom2, geom1_pts[i])
        dst1, dst2 = geom1_pts[i].distance(pt1), geom1_pts[i].distance(pt2)
        dst = max(dst1, dst2)
        if dst > dst_max: dst_max = dst
    return dst_max

def GetTangentCycle(geom1, geom2):
    eps, intersection, bldistance = 2e-3, MultiPoint(), 0
    while intersection.geom_type == "MultiPoint":
        bldistance += eps
        blgeom2 = LineString(geom2).offset_curve(-bldistance, join_style=1)
        intersection = geom1.intersection(blgeom2)
    return blgeom2, bldistance
    
def GetNextCycleContour(geom1, geom2, dst_delta):
    blgeom2, dst = LineString(geom2).offset_curve(-dst_delta, join_style=1), dst_delta
    spts, intersection_angles = GetSimpleOrderedIntersectionPoints(geom1, blgeom2)
    if len(spts) == 0:
        blgeom2, dst = GetTangentCycle(geom1, geom2)
        spts, intersection_angles = GetSimpleOrderedIntersectionPoints(geom1, blgeom2)
    return blgeom2, spts, dst

def GetPointOnLineWithDistance(line, distance):
    if distance < 0.0 or distance > line.length:
        return None
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return Point(coords[i][0], coords[i][1])
        if pd > distance:
           return line.interpolate(distance)
        
def GetSwingBrutingTrace(cycle_trace_pts, dst):
    blcycle = LineString(cycle_trace_pts).offset_curve(dst, join_style=1)
    return [Point(pt[0], pt[1]) for pt in blcycle.coords]

def GetSwingDiskTrace(trace_pts, R):
    blcycle = LineString(trace_pts).offset_curve(-R, join_style=1)
    return [Point(pt[0], pt[1]) for pt in blcycle.coords]

def GetCurrentShape(current_rough_pts, disk, angles, n):
    diamond_ring = LinearRing(current_rough_pts)
    if disk.intersects(diamond_ring):
        rough = sp.difference(Polygon(current_rough_pts), disk).exterior
        npts = diamond_ring.intersection(disk)
        if npts.geom_type == 'LineString':
            angle_edge1 = atan2(npts.coords[0][1], npts.coords[0][0])
            if angle_edge1 < 0: angle_edge1 += 2*pi
            angle_edge2 = atan2(npts.coords[-1][1], npts.coords[-1][0])
            if angle_edge2 < 0: angle_edge2 += 2*pi
        else:
            if npts.geom_type == 'Point':
                return current_rough_pts
            else:
                angle_edge1 = atan2(npts.geoms[-1].coords[0][1], npts.geoms[-1].coords[0][0])
                if angle_edge1 < 0: angle_edge1 += 2*pi
                angle_edge2 = atan2(npts.geoms[0].coords[-1][1], npts.geoms[0].coords[-1][0])
                if angle_edge2 < 0: angle_edge2 += 2*pi
        
        if angle_edge1 > angle_edge2: angle_edge2 += 2*pi
        for i in range(n):
            cur_angle = angles[i]
            if angle_edge1 <= cur_angle and cur_angle <= angle_edge2:
                line = LineString([(0,0), (30*cos(cur_angle), 30*sin(cur_angle))])
                current_rough_pts[i] = rough.intersection(line)
    return current_rough_pts

In [ ]:
# def GetBaloonedContourWithContainPercentage(geom1, geom2, angles, n, contain_percent):
#     delta = 1e-3
#     bldistance = delta

#     spts, intersection_angles = GetOrderedIntersectionPoints(geom1, geom2)
#     if len(spts) > 0:
#         indices, N = GetRotatedReperIndices(angles, intersection_angles)
#         geom2_pts = Contour2Points(geom2, intersection_angles[0], n)
#         contain_length = 0
#         for j in range(N):
#             i_min, i_max = indices[j], indices[(j+1)%N]
#             contain_length += LineString(geom2_pts[i_min:i_max]).length
#         current_contain_percent = contain_length / geom2.length
#     else:
#         current_contain_percent = 1

#     iter = 0
#     while current_contain_percent > contain_percent and iter < 1000:
#         prev_contain_percent = current_contain_percent
#         blgeom2 = GetBaloonedContour(geom2, bldistance)
#         spts, intersection_angles = GetOrderedIntersectionPoints(geom1, blgeom2)
#         if len(spts) > 0:
#             indices, N = GetRotatedReperIndices(angles, intersection_angles)
#             blgeom2_pts = Contour2Points(blgeom2, intersection_angles[0], n)
#             contain_length = 0
#             for j in range(0, N, 2):
#                 i_min, i_max = indices[j]-1, indices[(j+1)%N]-1
#                 contain_length += LineString(blgeom2_pts[i_min:i_max]).length
#             if abs(contain_length) > 1e-5:
#                 current_contain_percent = contain_length / blgeom2.length
#                 # print(contain_length, geom1.length)
#         if abs(prev_contain_percent-current_contain_percent) < 1e-3:
#             delta *= 2
#         elif abs(prev_contain_percent-current_contain_percent) > 1e-2:
#             delta /= 2
#         bldistance += delta
#         iter += 1
#         print(current_contain_percent, end="\r")
#     return blgeom2, spts, current_contain_percent

In [264]:
rough   = GetContourFromOBJ(rough_filename)
pear    = GetSmoothedPear(GetContourFromOBJ(diamond_filename))

angles = [2*pi*i/n for i in range(n)]
blpear, stps = GetBaloonedContourWithContainPercentage(rough, pear, n, contain_percent)
spts, intersection_angles = GetOrderedIntersectionPoints(rough, blpear)

# blpear, stps, real_contain_percent  = GetBaloonedContourWithContainPercentage(rough_pts, pear_pts, angles, n, contain_percent)
# blpear_pts = Contour2Points(blpear, 0, n)

spts, intersection_angles = GetOrderedIntersectionPoints(rough, blpear)
indices, N = GetReperIndices(angles, intersection_angles)

rough_pts = Contour2Points(rough, 0, n)
blpear_pts = Contour2Points(blpear, 0, n)

# plot_line(pear, color=(1,0,0), add_points=False, label="target pear")
# plot_line(blpear, color=(0.5,0.2,0),  add_points=False, label="balooned pear")
# plot_points(stps, color=(0,0,0),  markersize=5)
# plot_line(rough, color=(0,0,1),  add_points=False, label="rough pear")
# plot_points(Point(0,0), color=(1,0,0), label="diamond center")

# plt.rcParams["figure.figsize"] = (8.0, 8.0)
# # plt.title('contain_percent = ' + str(round(real_contain_percent*100, 6)))
# plt.grid(False)
# plt.legend()
# plt.show()

In [265]:
global_cycle_traces, global_sptss, global_dsts = [], [], []

# for j in range(0, 1, 2):
# for j in range(2, 3, 2):
for j in range(0, N, 2):
    cycle_traces, sptss, dsts = [], [], []
    i_min, i_max = indices[j], indices[j+1]

    cycle_rough = LineString(rough_pts[i_min:i_max])
    cycle_ground = LineString(blpear_pts[i_min:i_max])

    dst_max = GetMaxDistance(rough_pts[i_min:i_max], cycle_ground)
    cycle_count = floor(dst_max / max_delta_distance)

    cycle_trace, spts, dst = GetNextCycleContour(cycle_rough, cycle_ground, max_delta_distance)
    cycle_traces.append(cycle_trace)
    sptss.append(spts)
    dsts.append(dst)

    for k in range(cycle_count):
        cycle_trace, spts, dst = GetNextCycleContour(cycle_rough, cycle_traces[-1], max_delta_distance)
        cycle_traces.append(cycle_trace)
        sptss.append(spts)
        dsts.append(dst)
    
    cycle_traces.reverse()
    sptss.reverse()
    dsts.reverse()
    global_cycle_traces.append(cycle_traces)
    global_sptss.append(sptss)
    global_dsts.append(dsts)

In [266]:
final_cycle_traces = []

# for j in range(0, 1, 2):
# for j in range(2, 3, 2):
for j in range(0, N, 2):
    num = int(j/2)
    cycle_traces = []
    for k in range(len(global_cycle_traces[num])-1):
        current_trace = global_cycle_traces[num][k]
        current_trace_pts = [Point(pt[0], pt[1]) for pt in current_trace.coords]
        cut_pts = global_sptss[num][k+1]
        pt1, pt = nearest_points(current_trace, cut_pts[0])
        pt2, pt = nearest_points(current_trace, cut_pts[-1])

        pt1_min, dst_min = current_trace_pts[0], current_trace_pts[0].distance(pt1)
        for i in range(1, len(current_trace_pts)):
            dst = current_trace_pts[i].distance(pt1)
            if dst < dst_min:
                pt1_min, dst_min = current_trace_pts[i], dst

        pt2_min, dst_min = current_trace_pts[0], current_trace_pts[0].distance(pt2)
        for i in range(1, len(current_trace_pts)):
            dst = current_trace_pts[i].distance(pt2)
            if dst < dst_min:
                pt2_min, dst_min = current_trace_pts[i], dst

        assert current_trace.contains(pt1_min) == True
        assert current_trace.contains(pt2_min) == True
        
        cut_line_pts = [Point(pt[0], pt[1]) for pt in split(split(current_trace, pt1_min).geoms[1], pt2_min).geoms[0].coords]
        cycle_traces.append(cut_line_pts)

        # plot_points(pt1_min, color=(0,0,0), markersize=5)
        # plot_points(pt2_min, color=(0,0,0), markersize=5)
        # plot_line(current_trace, add_points=False, color=(0,0,1))
        # plot_line(cut_line, add_points=False, color=(0,1,0))
        # break
        
    cycle_traces.append([Point(pt[0], pt[1]) for pt in global_cycle_traces[num][-1].coords])
    final_cycle_traces.append(cycle_traces)

In [ ]:
# j = 0
# # print(indices[j+1]-indices[j])
# if j < N-1:
#     plot_points(blpear_pts[indices[j]:indices[j+1]], color=(0.5,0.2,0),  markersize=1)
#     plot_points(rough_pts[indices[j]:indices[j+1]], color=(0,0,1),  markersize=1)
# else:
#     plot_points(blpear_pts[indices[j]:], color=(0.5,0.2,0),  markersize=1)
#     plot_points(blpear_pts[:indices[0]], color=(0.5,0.2,0),  markersize=1)
#     plot_points(rough_pts[indices[j]:], color=(0,0,1),  markersize=1)
#     plot_points(rough_pts[:indices[0]], color=(0,0,1),  markersize=1)

plot_line(pear, color=(1,0,0), add_points=False, label="target pear")
plot_line(blpear, color=(0.5,0.2,0),  add_points=False, label="balooned pear")
plot_line(rough, color=(0,0,1),  add_points=False, label="rough pear")
plot_points(Point(0,0), color=(1,0,0), label="diamond center")

# for j in range(len(global_cycle_traces)):
#     for k in range(len(global_cycle_traces[j])):
#         plot_points(global_cycle_traces[j][k], color=(0,1,0), markersize=1)
#         plot_points(global_sptss[j][k], color=(0,0,0), markersize=3)
#         # print(len(list(global_cycle_traces[j][k].coords)))

for j in range(len(final_cycle_traces)):
    for k in range(len(final_cycle_traces[j])):
        plot_points(final_cycle_traces[j][k], color=(0,1,0), markersize=1)

# j, k = 0, 0
# plot_points(global_cycle_traces[j][k], color=(0,1,0), markersize=1)
# plot_points(global_sptss[j][k], color=(0,0,0), markersize=3)

# j, k = 0, 0
# plot_points(final_cycle_traces[j][k], color=(0,1,0), markersize=1)

plt.grid(False)
plt.legend()
plt.show()

In [209]:
# blpear_ground_pts = [blpear_pts[indices[j]:indices[j+1]] for j in range(0, N, 2)]
# notmoved_bruting_traces, notmoved_disk_traces, all_dsts, cycle_counts = [], [], [], []
# for j in range(len(final_cycle_traces)):

#     bruting_trace, disk_trace, dsts, cycle_ccount = [], [], [], []

#     local_cycle_count = floor(global_dsts[j][0] / disk_delta_translation)
#     cycle_ccount.append(local_cycle_count)

#     for i in range(local_cycle_count):
#         trace = final_cycle_traces[j][0]
#         bltrace = [Point(pt[0], pt[1]) for pt in LineString(trace).offset_curve(-R, join_style=1).coords]
#         bruting_trace.append(trace)
#         disk_trace.append(bltrace)
#         dsts.append(disk_delta_translation)

#     bruting_trace.append(final_cycle_traces[j][1])
#     bltrace = [Point(pt[0], pt[1]) for pt in LineString(trace).offset_curve(-R, join_style=1).coords]
#     disk_trace.append(bltrace)
#     dsts.append(global_dsts[j][0]-local_cycle_count*disk_delta_translation)

#     local_cycle_count = floor(max_delta_distance / disk_delta_translation)

#     for k in range(1, len(final_cycle_traces[j])-1):

#         for i in range(local_cycle_count):
#             trace = final_cycle_traces[j][k]
#             bltrace = [Point(pt[0], pt[1]) for pt in LineString(trace).offset_curve(-R, join_style=1).coords]
#             bruting_trace.append(trace)
#             disk_trace.append(bltrace)
#             dsts.append(disk_delta_translation)

#         bruting_trace.append(final_cycle_traces[j][k+1])
#         bltrace = [Point(pt[0], pt[1]) for pt in LineString(trace).offset_curve(-R, join_style=1).coords]
#         disk_trace.append(bltrace)
#         dsts.append(disk_delta_translation)

#         cycle_ccount.append(local_cycle_count)
    
#     for i in range(local_cycle_count):
#         trace = final_cycle_traces[j][-1]
#         bltrace = [Point(pt[0], pt[1]) for pt in LineString(trace).offset_curve(-R, join_style=1).coords]
#         bruting_trace.append(trace)
#         disk_trace.append(bltrace)
#         dsts.append(disk_delta_translation)
    
#     cycle_ccount.append(local_cycle_count+1)

#     notmoved_bruting_traces.append(bruting_trace)
#     notmoved_disk_traces.append(disk_trace)
#     all_dsts.append(dsts)
#     cycle_counts.append(cycle_ccount)

# for j in range(len(notmoved_bruting_traces)):
#     for k in range(len(notmoved_bruting_traces[j])):
#         if k%2 == 1: notmoved_bruting_traces[j][k].reverse(); notmoved_disk_traces[j][k].reverse()

In [267]:
blpear_ground_pts = [blpear_pts[indices[j]:indices[j+1]] for j in range(0, N, 2)]
bruting_traces, disk_traces, all_dsts, cycle_counts = [], [], [], []
# bruting_traces, disk_traces = [], []
for j in range(len(final_cycle_traces)):

    bruting_trace, disk_trace, dsts, cycle_ccount = [], [], [], []

    # dst_max = GetMaxDistance(final_cycle_traces[j][0], LineString(final_cycle_traces[j][1]))
    # local_cycle_count = floor(dst_max / disk_delta_translation)
    local_cycle_count = floor(global_dsts[j][0] / disk_delta_translation)
    cycle_ccount.append(local_cycle_count)

    for i in range(local_cycle_count):
        trace = GetSwingBrutingTrace(final_cycle_traces[j][0], disk_delta_translation*(i+1))
        bltrace = GetSwingDiskTrace(trace, R)
        bruting_trace.append(trace)
        disk_trace.append(bltrace)
        dsts.append(disk_delta_translation)

    dsts.append(global_dsts[j][0]-local_cycle_count*disk_delta_translation)
    bruting_trace.append(final_cycle_traces[j][1])
    bltrace = GetSwingDiskTrace(final_cycle_traces[j][1], R)
    disk_trace.append(bltrace)

    local_cycle_count = floor(max_delta_distance / disk_delta_translation)

    for k in range(1, len(final_cycle_traces[j])-1):

        for i in range(local_cycle_count):
            trace = GetSwingBrutingTrace(final_cycle_traces[j][k], disk_delta_translation*(i+1))
            bltrace = GetSwingDiskTrace(trace, R)
            bruting_trace.append(trace)
            disk_trace.append(bltrace)
            dsts.append(disk_delta_translation)

        bruting_trace.append(final_cycle_traces[j][k+1])
        bltrace = GetSwingDiskTrace(final_cycle_traces[j][k+1], R)
        disk_trace.append(bltrace)
        dsts.append(disk_delta_translation)

        cycle_ccount.append(local_cycle_count)
    
    for i in range(local_cycle_count):
        trace = GetSwingBrutingTrace(final_cycle_traces[j][-1], disk_delta_translation*(i+1))
        bltrace = GetSwingDiskTrace(trace, R)
        bruting_trace.append(trace)
        disk_trace.append(bltrace)
        dsts.append(disk_delta_translation)

    bruting_trace.append(blpear_ground_pts[j])
    bltrace = GetSwingDiskTrace(blpear_ground_pts[j], R)
    disk_trace.append(bltrace)
    dsts.append(disk_delta_translation)

    cycle_ccount.append(local_cycle_count+1)

    bruting_traces.append(bruting_trace)
    disk_traces.append(disk_trace)
    all_dsts.append(dsts)
    cycle_counts.append(cycle_ccount)

for j in range(len(bruting_traces)):
    for k in range(len(bruting_traces[j])):
        if k%2 == 1: bruting_traces[j][k].reverse(); disk_traces[j][k].reverse()

In [ ]:
plot_line(pear, color=(1,0,0), add_points=False, label="target pear")
plot_line(blpear, color=(0.5,0.2,0),  add_points=False, label="balooned pear")
plot_line(rough, color=(0,0,1),  add_points=False, label="rough pear")
plot_points(Point(0,0), color=(1,0,0), label="diamond center")

# for j in range(len(final_cycle_traces)):
#     for k in range(len(final_cycle_traces[j])):
#         plot_points(final_cycle_traces[j][k], color=(1,0,0), markersize=1, alpha=0.5)

# for j in range(len(bruting_traces)):
#     for k in range(len(bruting_traces[j])):
#         plot_points(bruting_traces[j][k], color=(0,1,0), markersize=1)

# for j in range(len(disk_traces)):
#     for k in range(len(disk_traces[j])):
#         plot_points(disk_traces[j][k], color=(1,0,1), markersize=1)

# print(cycle_counts[1][1])

# for j in range(len(notmoved_bruting_traces)):
#     for k in range(len(notmoved_bruting_traces[j])):
#         plot_points(notmoved_bruting_traces[j][k], color=(0,1,0), markersize=1)

# for j in range(len(notmoved_disk_traces)):
#     for k in range(len(notmoved_disk_traces[j])):
#         plot_points(notmoved_disk_traces[j][k], color=(1,0,1), markersize=1)

plot_points(notmoved_bruting_traces[1][7], color=(0,1,0), markersize=1)

plt.grid(False)
plt.legend()
# plt.xlim(1.5,3)
# plt.ylim(0.5,3.5)
plt.xlim(-3,-1)
plt.ylim(-3.5,-0.5)
plt.show()

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)
current_rough_pts = Contour2Points(rough, 0, n)

plot_line(pear, color=(1,0,0), add_points=False, label="target pear")
plot_line(blpear, color=(0.5,0.2,0),  add_points=False, label="balooned pear")
# plot_line(rough, color=(0,0,1),  add_points=False, label="rough pear")
plot_points(Point(0,0), color=(1,0,0), label="diamond center")

# j, k, l = 0, 0, 0
# j, k, l = len(disk_traces)-1, len(disk_traces[j])-1, len(disk_traces[j][k])-1
j, k, l = 0, 0, 100
print(j, k, l)

# plot_points(bruting_traces[j][k], color=(0,1,0), markersize=1, label="bruting trace")
# plot_points(disk_traces[j][k], color=(1,0,1), markersize=1, label="disk trace")

disk = transform(lambda x, y, z=None: (x+disk_traces[j][k][l].x, y+disk_traces[j][k][l].y), original_disk)
plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)

for p in range(k-1):
    for q in tqdm(range(len(disk_traces[j][p]))):
        disk = transform(lambda x, y, z=None: (x+disk_traces[j][p][q].x, y+disk_traces[j][p][q].y), original_disk)
        current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
for q in tqdm(range(l)):
    disk = transform(lambda x, y, z=None: (x+disk_traces[j][k][q].x, y+disk_traces[j][k][q].y), original_disk)
    current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
    
plot_points(current_rough_pts, color=(0,0,1), markersize=1, label="rough pear")

plt.grid(False)
plt.legend()
plt.xlim(2,3)
plt.ylim(1,3)
plt.show()

In [ ]:
disk = Point(0,0).buffer(R, quad_segs=n)

j, k, l = 0, 0, 0

angle = matan(disk_traces[j][k][l])
xval = R+r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x

plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(pear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
          color=(1,0,0), add_points=False, label="target pear")
plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(blpear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
          color=(0.5,0.2,0),  add_points=False, label="balooned pear")
plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(rough, -angle, origin=Point(0.0,0.0), use_radians=True)), 
          color=(0,0,1),  add_points=False, label="rough pear")
plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(Point(0,0), -angle, origin=Point(0.0,0.0), use_radians=True)), 
            color=(1,0,0), label="diamond center")

plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(bruting_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
            color=(0,1,0), markersize=1, label="bruting trace")
plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(disk_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
            color=(1,0,1), markersize=1, label="disk trace")

plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(bruting_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
            color=(0,1,0), markersize=5, label="touch point")
plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
            color=(1,0,1), markersize=5, label="disk center")

disk = transform(lambda x, y, z=None: (x+R+r, y), Polygon(disk))
plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)

plt.rcParams["figure.figsize"] = (8.0, 8.0)
plt.grid(True)
plt.legend()
plt.xlim(-6, 6)
plt.ylim(-6, 6)
plt.show()

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)

draw_path = "algo/big/transformed/0/pics/"
movie_path = "algo/big/transformed/0/"
current_rough_pts = Contour2Points(rough, 0, n)
count = 0

j = 0
xstep = 0
for k in tqdm(range(len(disk_traces[j]))):
    for l in range(0, len(disk_traces[j][k]), framerate):
        angle = atan2(disk_traces[j][k][l].y, disk_traces[j][k][l].x)
        xstep += (1 / len(disk_traces[j][k])) * all_dsts[j][k]
        print(xstep, end="\r")
        xval = r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep

        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(pear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(1,0,0), add_points=False, label="target pear")
        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(blpear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(0.5,0.2,0),  add_points=False, label="balooned pear")
        # plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(rough, -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #         color=(0,0,1),  add_points=False, label="rough pear")
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(Point(0,0), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(1,0,0), label="diamond center")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(bruting_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=1, label="bruting trace")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(disk_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=1, label="disk trace")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(bruting_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=5, label="touch point")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=5, label="disk center")

        disk = transform(lambda x, y, z=None: (x+r-xstep, y), original_disk)
        plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)

        disk = transform(lambda x, y, z=None: (x+disk_traces[j][k][l].x, y+disk_traces[j][k][l].y), original_disk)
        current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
    
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(MultiPoint(current_rough_pts), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(0,0,1), markersize=1, label="rough pear")

        plt.rcParams["figure.figsize"] = (8.0, 8.0)
        plt.grid(True)
        plt.legend()
        plt.xlim(-6, 6)
        plt.ylim(-6, 6)
        plt.savefig(draw_path + str(count) + ".png")
        plt.close()
        count += 1

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)

draw_path = "algo/big/test/pics/"
count = 0

current_rough_pts = Contour2Points(rough, 0, n)

j = 1
xstep = 0
for k in tqdm(range(len(disk_traces[j]))):
    for l in tqdm(range(0, len(disk_traces[j][k]), framerate)):
        angle = atan2(disk_traces[j][k][l].y, disk_traces[j][k][l].x)
        xstep += (framerate / len(disk_traces[j][k])) * all_dsts[j][k]
        xval = r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep

        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(pear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(1,0,0), add_points=False, label="target pear")
        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(blpear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(0.5,0.2,0),  add_points=False, label="balooned pear")
        # plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(rough, -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #         color=(0,0,1),  add_points=False, label="rough pear")
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(Point(0,0), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(1,0,0), label="diamond center")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(bruting_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=1, label="bruting trace")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(disk_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=1, label="disk trace")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(bruting_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=5, label="touch point")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=5, label="disk center")

        disk = transform(lambda x, y, z=None: (x+r-xstep, y), original_disk)
        plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)

        xx, yy = disk_traces[j][k][l].x, disk_traces[j][k][l].y
        # nrm = sqrt(xx**2+yy**2)
        # xxx, yyy = (nrm-xstep)*xx/nrm, (nrm-xstep)*yy/nrm
        disk = transform(lambda x, y, z=None: (x+xx, y+yy), original_disk)
        current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
    
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(MultiPoint(current_rough_pts), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(0,0,1), markersize=1, label="rough pear")
        # plot_points(affinity.rotate(transform(lambda x, y, z=None: (x+(xval), y),MultiPoint(mod_current_rough_pts)), -angle, origin=Point(0.0,0.0), use_radians=True), 
        #             color=(0,0,1), markersize=1, label="rough pear")

        plt.rcParams["figure.figsize"] = (8.0, 8.0)
        plt.grid(True)
        plt.legend()
        plt.xlim(-6, 6)
        plt.ylim(-6, 6)
        plt.savefig(draw_path + str(count) + ".png")
        plt.close()
        count += 1

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)

draw_path = "algo/big/test/pics/"
count = 0

# current_rough_pts = Contour2Points(rough, 0, n)

j = 1
xstep = 0
# for k in tqdm(range(cycle_counts[j][p])):
for k in tqdm(range(38, 44)):
    for l in tqdm(range(0, len(disk_traces[j][k]), framerate)):
        angle = atan2(disk_traces[j][k][l].y, disk_traces[j][k][l].x)
        xstep += (framerate / len(disk_traces[j][k])) * all_dsts[j][k]
        xval = r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep

        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(pear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(1,0,0), add_points=False, label="target pear")
        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(blpear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(0.5,0.2,0),  add_points=False, label="balooned pear")
        # plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(rough, -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #         color=(0,0,1),  add_points=False, label="rough pear")
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(Point(0,0), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(1,0,0), label="diamond center")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(bruting_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=1, label="bruting trace")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(disk_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=1, label="disk trace")

        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(bruting_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=5, label="touch point")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=5, label="disk center")

        disk = transform(lambda x, y, z=None: (x+r-xstep, y), original_disk)
        plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)

        disk = transform(lambda x, y, z=None: (x+disk_traces[j][k][l].x, y+disk_traces[j][k][l].y), original_disk)
        current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
    
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(MultiPoint(current_rough_pts), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(0,0,1), markersize=1, label="rough pear")
        # plot_points(affinity.rotate(transform(lambda x, y, z=None: (x+(xval), y),MultiPoint(mod_current_rough_pts)), -angle, origin=Point(0.0,0.0), use_radians=True), 
        #             color=(0,0,1), markersize=1, label="rough pear")

        plt.rcParams["figure.figsize"] = (8.0, 8.0)
        plt.grid(True)
        plt.legend()
        plt.xlim(-6, 6)
        plt.ylim(-6, 6)
        plt.savefig(draw_path + str(count) + ".png")
        plt.close()
        count += 1

movie_path = "algo/big/test/"
with imageio.get_writer(movie_path+'movie8.gif', mode='I', duration=0) as writer:
    for j in tqdm(range(count)):
        writer.append_data(imageio.v2.imread(draw_path + str(j) + ".png"))

In [ ]:
movie_path = "algo/big/test/"
with imageio.get_writer(movie_path+'movie0.gif', mode='I', duration=0) as writer:
    for j in tqdm(range(count)):
        writer.append_data(imageio.v2.imread(draw_path + str(j) + ".png"))

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)
current_rough_pts = Contour2Points(rough, 0, n)

xvals = []
rangles = []
dvals = []
time = []

j, l = 1, 0
bruting_time = 5.0

whole_length = 0
# for k in range(cycle_counts[j][l]):
for k in range(len(notmoved_disk_traces[j])):
    whole_length += len(notmoved_disk_traces[j][k])

# j, l = 1, 1
# bruting_time = 10.0

# print(cycle_counts[j])

# whole_length = 0
# for k in range(cycle_counts[j][l-1], cycle_counts[j][l-1]+cycle_counts[j][l]):
#     whole_length += len(notmoved_disk_traces[j][k])

# cur_length, xstep = 0, 0
# for k in range(cycle_counts[j][l-1], cycle_counts[j][l]+1):
#     for l in range(len(disk_traces[j][k])):
#         cur_length += 1
#         angle = atan2(disk_traces[j][k][l].y, disk_traces[j][k][l].x)
#         xstep += (1 / len(disk_traces[j][k])) * all_dsts[j][k]
#         xval = r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep
#         dval = r-xstep

#         xvals.append(xval)
#         rangles.append(angle*180/pi)
#         dvals.append(dval)
#         time.append(bruting_time*cur_length/whole_length)

cur_length, xstep = 0, 0
# for k in range(cycle_counts[j][l]):
# for k in range(cycle_counts[j][l-1], cycle_counts[j][l-1]+cycle_counts[j][l]):
for k in range(len(notmoved_disk_traces[j])):
    for l in range(len(notmoved_disk_traces[j][k])):
        cur_length += 1
        angle = atan2(notmoved_disk_traces[j][k][l].y, notmoved_disk_traces[j][k][l].x)
        xstep += (1 / len(notmoved_disk_traces[j][k])) * all_dsts[j][k]
        xval = r-affinity.rotate(notmoved_disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep
        dval = r-xstep

        xvals.append(xval)
        rangles.append(angle*180/pi)
        dvals.append(dval)
        time.append(bruting_time*cur_length/whole_length)

drangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, rangles[1:], rangles)]
dxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, xvals[1:], xvals)]
ddrangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, drangles[1:], drangles)]
ddxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dxvals[1:], dxvals)]

fig, axs = plt.subplots(5, 1)
fig.set_size_inches(10.0, 15.0)

axs[0].plot(time, rangles, marker='.', color="red", markersize=0.1)
axs[0].set_ylabel('rotation angle of diamond')
axs[0].grid(True)

axs[1].plot(time[:-2], ddrangles, marker='.', color="red", markersize=0.1)
axs[1].plot(time[:-2], [1760]*(len(time)-2), marker='.', color="blue", markersize=0.1)
axs[1].plot(time[:-2], [-1760]*(len(time)-2), marker='.', color="blue", markersize=0.1)
axs[1].set_ylabel('rotation acceleration of diamond')
axs[1].set_ylim([-2000, 2000])
axs[1].grid(True)

axs[2].plot(time, xvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[2].set_ylabel('ordinate of diamond center')
axs[2].grid(True)

axs[3].plot(time[:-2], ddxvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[3].plot(time[:-2], [15]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
axs[3].plot(time[:-2], [-15]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
axs[3].set_ylabel('acceleration of diamond center')
axs[3].set_ylim([-20, 20])
axs[3].grid(True)

axs[4].plot(time, dvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[4].set_xlabel('time')
axs[4].set_ylabel('ordinate of disk center')
axs[4].grid(True)

In [ ]:
original_disk = Point(0,0).buffer(R, quad_segs=n)
current_rough_pts = Contour2Points(rough, 0, n)

xvals = []
rangles = []
dvals = []
time = []

j = 1
# bruting_time, k_min, k_max = 12, 0, 4
# bruting_time, k_min, k_max = 27, 4, 10
# bruting_time, k_min, k_max = 36, 10, 16
# bruting_time, k_min, k_max = 45, 16, 22
# bruting_time, k_min, k_max = 54, 22, 28
# bruting_time, k_min, k_max = 63, 28, 34
# bruting_time, k_min, k_max = 78, 34, 40
# bruting_time, k_min, k_max = 87, 40, 46
# bruting_time, k_min, k_max = 93, 46, 53
bruting_time, k_min, k_max = 750, 0, 53
whole_length = 0
for k in range(k_min, k_max):
    whole_length += len(disk_traces[j][k])

cur_length, xstep = 0, 0
for k in range(k_min, k_max):
    for l in range(len(disk_traces[j][k])):
        cur_length += 1
        angle = atan2(disk_traces[j][k][l].y, disk_traces[j][k][l].x)
        xstep += (1 / len(disk_traces[j][k])) * all_dsts[j][k]
        xval = r-affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x-xstep
        dval = r-xstep

        xvals.append(xval)
        rangles.append(angle*180/pi)
        dvals.append(dval)
        time.append(bruting_time*cur_length/whole_length)

drangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, rangles[1:], rangles)]
dxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, xvals[1:], xvals)]
ddrangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, drangles[1:], drangles)]
ddxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dxvals[1:], dxvals)]

fig, axs = plt.subplots(7, 1)
fig.set_size_inches(12.0, 21.0)

axs[0].plot(time, rangles, marker='.', color="red", markersize=0.1)
axs[0].set_ylabel('rotation angle of diamond')
axs[0].grid(True)

axs[1].plot(time[:-1], drangles, marker='.', color="red", markersize=0.1)
axs[1].set_ylabel('rotation velocity of diamond')
# axs[1].set_ylim([-2000, 2000])
axs[1].grid(True)

axs[2].plot(time[:-2], ddrangles, marker='.', color="red", markersize=0.1)
axs[2].plot(time[:-2], [1760]*(len(time)-2), marker='.', color="blue", markersize=0.1)
axs[2].plot(time[:-2], [-1760]*(len(time)-2), marker='.', color="blue", markersize=0.1)
axs[2].set_ylabel('rotation acceleration of diamond')
axs[2].set_ylim([-2000, 2000])
axs[2].grid(True)

axs[3].plot(time, xvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[3].set_ylabel('ordinate of diamond center')
axs[3].grid(True)

axs[4].plot(time[:-1], dxvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[4].set_ylabel('velocity of diamond center')
# axs[4].set_ylim([-20, 20])
axs[4].grid(True)

axs[5].plot(time[:-2], ddxvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[5].plot(time[:-2], [15]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
axs[5].plot(time[:-2], [-15]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
axs[5].set_ylabel('acceleration of diamond center')
axs[5].set_ylim([-20, 20])
axs[5].grid(True)

axs[6].plot(time, dvals, marker='.', linestyle="-", color="red", markersize=0.1)
axs[6].set_xlabel('time')
axs[6].set_ylabel('ordinate of disk center')
axs[6].grid(True)

In [ ]:
for j in range(len(xvals)):
    for q in range(len(xvals[j])):
        test = TestTrace(time[j][q], rangles[j][q], xvals[j][q], dvals[j][q], j, q, False,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval)
        print(test)

In [ ]:
xvals, rangles, dvals, time, delta, delta_time = [], [], [], [], [], []
for j in range(len(result_disk_traces)):
    local_xvals, local_rangles, local_dvals, local_time, local_delta, local_delta_time = [], [], [], [], [], []
    for q in range(0, len(result_disk_traces[j]), cycle_count):
        local_local_xvals, local_local_rangles, local_local_dvals, local_local_time, local_local_delta = [], [], [], [], []
        whole_length, cur_length, xstep = 0, 0, 0
        for k in range(cycle_count): whole_length += len(result_disk_traces[j][q+k])
        whole_length -= (cycle_count-1)
        delta_ttime, delta_xstep = bruting_time/whole_length, 1/whole_length
        for k in range(cycle_count):
            if k > 0:
                angle = atan2(result_disk_traces[j][q+k][0].y, result_disk_traces[j][q+k][0].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][0], -angle, origin=Point(0.0,0.0), use_radians=True).x
                dlt, km = local_local_xvals[-1]-xval, 1
            else:
                dlt, km = 0, 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                angle = atan2(result_disk_traces[j][q+k][l].y, result_disk_traces[j][q+k][l].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x +dlt
                local_local_xvals.append(xval); local_local_rangles.append(angle*180/pi)
                local_local_time.append(bruting_time*cur_length/whole_length)
                cur_length += 1
            local_local_delta.append(dlt)
        for k in range(cycle_count):
            km = 1 if k > 0 else 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                xstep += delta_xstep * (local_local_delta[1]*1)
                local_local_dvals.append(cycle_count*xstep); 
        for m in range(len(local_local_xvals)):
            local_local_xvals[m] -= local_local_dvals[m]   
        local_xvals.append(local_local_xvals); local_rangles.append(local_local_rangles); 
        local_dvals.append(local_local_dvals); local_time.append(local_local_time); 
        local_delta.append(local_local_delta); local_delta_time.append(delta_ttime)
    xvals.append(local_xvals); rangles.append(local_rangles); dvals.append(local_dvals); time.append(local_time)
    delta.append(local_delta); delta_time.append(local_delta_time)